In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
from clasificacion_chinchetas import *
import tqdm
import numpy as np
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import sys
from shapely.geometry import mapping
from shapely import geometry
import cv2
import numpy as np

## Modelo de deteccion de objetos basado en yolov7

In [ ]:
Modelo=modelo(weights=r"best_.pt")

## Modelo clasificador basado en Alexnet

In [ ]:
class alexnet1():
    def __init__(self,weights,num_classes,idx_to_class):
        """inicializa el model, con los pesos entrenados"""
        alexnet=models.alexnet(pretrained=True)
        self.device = torch.device(0 if torch.cuda.is_available() else "cpu")
        checkpoint=torch.load(weights,map_location=self.device)
#         alexnet.features[1]= nn.Hardtanh()
#         alexnet.classifier[6] = nn.Linear(4096, num_classes)
#         alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))
        # alexnet
        alexnet.features[1]= nn.Hardtanh()
        alexnet.classifier[6] = nn.Linear(4096, 4096)
        alexnet.classifier.add_module("7",nn.Softplus())
        alexnet.classifier.add_module("8", nn.Linear(4096, 4096))
        alexnet.classifier.add_module("9",nn.Softplus())
        alexnet.classifier.add_module("10", nn.Linear(4096, 2048))
        alexnet.classifier.add_module("11", nn.Softplus())
        alexnet.classifier.add_module("12", nn.Linear(2048, num_classes))
        alexnet.classifier.add_module("13", nn.Softplus())
        alexnet.classifier.add_module("14",  nn.LogSoftmax(dim = 1))
        # for param in alexnet.parameters():
        #     param.requires_grad = False
        # alexnet.classifier[6] = nn.Linear(4096, num_classes)
        # alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))
        alexnet.load_state_dict(checkpoint['model_state_dict'])
        summary(alexnet, (3, 224, 224))
        self.model=alexnet
        self.idx_to_class=idx_to_class
    
    def predict_file(self,file,pad=True):
        """Genera prediccion sobre archivo"""
#         x = Image.open(file)
#         x = np.asarray(x)
#         x=np.stack([x[:,:,0],x[:,:,1],x[:,:,2]], axis=-1)
        x=cv2.imread(file)
        if pad:
            x=padding(x)
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        x=x/255*2-1
        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase 
    
    def predict_image(self,image,pad=True):
        """Generar predeccion de clase sobre imagen precargada"""
#         x = np.asarray(image)
#         x=np.stack([x[:,:,2],x[:,:,1],x[:,:,2]], axis=-1)
        x=np.array(image)
        if pad:
            x=padding(x)
        imagen=x.copy()
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        x=x/255*2-1
        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase, imagen 

In [ ]:
weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\5clases\best_modconstrucciones_2.pth"
num_classes=5
diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
model_class=alexnet1(weights=weights,num_classes=num_classes,idx_to_class=diciconario)
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

## Parametros del raster

In [ ]:
from detect_Alberto_v4 import Parametro_raster
raster=r".tif"

alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=100)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

## Delimitacion por shape de municipio

In [ ]:
def shape_transform(shape):
    """Convierte el shape en dataframe de coordenadas que engloba el polygon del shape para delimitar el raster"""
    c=[]
    angulo_manzana=[]

    # for manzana in range(len(shape)):
    #     proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
    #     angulos=[]
    #     d=[]
    #     poly=pd.DataFrame(proyecciones1[0])
    #     for point in range(1,len(poly)):
    #         d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
    #         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    #     angulo_manzana.append(angulos[d.index(max(d))])
    shape["angulo_manzana"]=0
    shape["geometry"]=shape["geometry"].envelope
    for manzana in range(len(shape)):
        proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(','.join([str(min_y),str(min_x),str(max_y),str(max_x)]))
    shape1=pd.DataFrame()
    shape1['points']=c
    shape1=shape1['points'].str.split(',',expand=True)
    shape1=shape1.astype({0:'float64',1:'float64',2:'float64',3:'float64'})
    shape1["cve_cat"]=shape["cve_cat"]
    shape1["angulo_manzana"]=shape["angulo_manzana"]
    shape=shape1
    return shape

## Generar prediccion sobre raster(municipio) con salida en shp

### Implementar modelos combinados

In [ ]:
postproceso(Modelo,'model_class',casas,conf_casas,
            clase_casas,terreno,conf_terreno,clase_terreno,
            clase_chinchetas, umbrales,sizes,
            raster,ancho,alto,dim,minx,maxx,miny,maxy,shape,angulo_get=0.000001,
            opt_conf_thres=0.2,imshow=False,imsave=False,path="C:/Users/ASUS/salida/imagen",
            clasificar_casas=False, clasificar_chinchetas=False)

## Generar imagenes mosaico

In [ ]:
def create_cuadro(alto,ancho,j,i):
    cuadro=[]
    for k in range(2):
        for l in range(2):
            cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                            maxy-(maxy-miny)/alto*(i+l)))
    cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
    for punto in cuadro:
        x=float(punto[0])
        y=float(punto[1])
    return(cuadro)

In [ ]:

from IPython.display import clear_output
def normalizar_array(img):
    
    return (img-np.min(img))/(np.max(img)-np.min(img))

### Multi_cuts

In [ ]:
# Path donde se guardan images
import tqdm as tqdm
from detect_Alberto_v4 import Parametro_raster
import numpy as np

path_mosaico_input=r"Mosaicos_input"
path_mosaico_output=r"Mosaicos_output"
raster=r"folder\corte_ESTE_ATLACOMULCO_1_satellite.tif"
raster2=r"folder\corte_ESTE_ATALCOMULCO_0.tif"
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=100)
# alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy
src = rasterio.open(raster)
print(src.crs)
src.meta.update(nodata=-3000)
generar_imagenes=int(input("Generar imagenes: 1 si, 0 no "))
if generar_imagenes==1:
    generar_imagenes_sinrotar=int(input("Generar imagenes sin rotar: 1 si, 0 no "))
    generar_imagenes_rotadas=int(input("Generar imagenes rotadas: 1 si, 0 no "))
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(1,ancho-1):#ancho
            for i in (range(1,alto-1)):#alto
                generar=0
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                cuadro= create_cuadro(alto,ancho,j,i)
                    # if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                    #     generar=1             
                # if generar==1:
                shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                four_images=[array[2],array[1],array[0]]
                # bin_image= [array]
                
                imagen_n = np.stack(four_images, axis=-1)
                # imagen_n = array[0]
                if generar_imagenes_sinrotar==1:
                    src2 = rasterio.open(raster2)
                    # src2.meta.update(nodata=0)
                    alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster2,metros=100)
                    cuadro2= create_cuadro(alto,ancho,j,i)
                    shapes2=[{"type":'Polygon','coordinates':[cuadro2]}]
                    array2, out_transform2 = rasterio.mask.mask(src2, shapes2, crop=True)
                    # four_images=[array2[2],array2[1],array2[0]]
                    # array2_0=normalizar_array(array2[0])
                    # array2_0=array2[0][array2[0]==0]
                    # array2_1=normalizar_array(array2[0][array2[0]!=0])
                    # array_f=np.stack([array2_0,array2_1], axis=0)
                    img=array2[0]
                    if np.sum(img)>10:
                        mini=np.min(img[img!=0])
                        img[img==0]=mini
                        imagen_n2=normalizar_array(img)
                        imagen_n2= np.array(imagen_n2*255, dtype='uint8')
                        imagen_n2=np.stack([imagen_n2,imagen_n2,imagen_n2],axis=-1)
                        # imagen_n2 = array2[0]array_f
                        cv2.imwrite(path_mosaico_input+"\\"+nameimg+".png",imagen_n)
                        
                        cv2.imwrite(path_mosaico_output+"\\"+nameimg+".png",imagen_n2)
                    else:
                        continue
                if generar_imagenes_rotadas==1:
                    angulo_1,imagen_ro=correct_orientation(imagen_n,dim)
                    cv2.imwrite(path_mosaico_salida+nameimg+"_"+str(angulo_1)+".png",imagen_ro)
                clear_output(wait=True)
                pbar.update(1)